In [1]:
#Importing Data From Kaggle
import os
os.environ['KAGGLE_USERNAME']="oa341791"
os.environ['KAGGLE_KEY']="e1d6525f3a295531cdd1ed595b5f8408"
!kaggle competitions download -c nn23-sports-image-classification

 94% 246M/262M [00:01<00:00, 220MB/s]
100% 262M/262M [00:01<00:00, 227MB/s]


In [2]:
#Extract Data
from zipfile import ZipFile
file_name="/content/nn23-sports-image-classification.zip"
with ZipFile(file_name,'r')as zip:
  zip.extractall()
  print("done")

done


In [3]:
# Importing Needed Libraries 
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [4]:
# Assigning Data to Categories and Reading It
CATEGORIES = ['Basketball','Football','Rowing','Swimming','Tennis','Yoga']
trainingData = []
path = '/content/Train'
for img in os.listdir(path):
    try:
      image = cv2.imread(os.path.join(path,img),cv2.IMREAD_UNCHANGED)
      if img.split('_')[0]=='Basketball' :
        imageLabel = 0
      elif img.split('_')[0]=='Football':
        imageLabel = 1
      elif img.split('_')[0]=='Rowing':
        imageLabel = 2
      elif img.split('_')[0]=='Swimming':
        imageLabel = 3
      elif img.split('_')[0]=='Tennis':
        imageLabel = 4
      elif img.split('_')[0]=='Yoga':
        imageLabel = 5
      imageResized = cv2.resize(image,(150,150))
      trainingData.append([imageResized,imageLabel])
    except Exception as e: 
      pass

In [5]:
# Splitting The Training Data into Labels and Features
X = []
Y = []
for features,labels in trainingData:
  X.append(features)
  Y.append(labels)
X = np.array(X)
Y = np.array(Y)
X = X/255

In [6]:
# Splitting Data into Training and Validation
from sklearn.model_selection import train_test_split
X_Train,X_Valid,Y_Train,Y_Valid=train_test_split(X,Y,test_size=0.2,shuffle=True)

In [7]:
# Perform Random Over Sampling to solve Class Imbalance
'''
from imblearn.over_sampling import RandomOverSampler
reshaped_X = X_Train.reshape(X_Train.shape[0],-1)
#oversampling
oversample = RandomOverSampler()
X_Train, Y_Train  = oversample.fit_resample(reshaped_X , Y_Train)

# reshaping X back to the first dims
#X_Train = X_Train.reshape(-1,224,224,3)
'''

'\nfrom imblearn.over_sampling import RandomOverSampler\nreshaped_X = X_Train.reshape(X_Train.shape[0],-1)\n#oversampling\noversample = RandomOverSampler()\nX_Train, Y_Train  = oversample.fit_resample(reshaped_X , Y_Train)\n\n# reshaping X back to the first dims\n#X_Train = X_Train.reshape(-1,224,224,3)\n'

In [8]:
# Importing Libraries Needed
import tensorflow.compat.v2 as tf
from keras import backend
from keras.applications import imagenet_utils
from keras.engine import training
from keras.layers import VersionAwareLayers
from keras.utils import data_utils
from keras.utils import layer_utils
from tensorflow.python.util.tf_export import keras_export
layers = VersionAwareLayers()
# Needed Weights for Transfer Learning
TF_WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"
)
def Xception(
    # Determining The Required Arguments and Their Default Values
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
):
    # Determine The Input Shape and its default
    input_shape = imagenet_utils.obtain_input_shape(
        input_shape,
        default_size=299,
        min_size=71,
        data_format=backend.image_data_format(),
        require_flatten=include_top,
        weights=weights,
    )
    #Entry Flow - changing Dimensionality through Strides
    # As we will not apply Input Tesnor then The First layer is Input Shape
    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    channel_axis = 1 if backend.image_data_format() == "channels_first" else -1
    # Using Separable Convolutions to perform depth-wise convolutions
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), use_bias=False, name="block1_conv1")(img_input)
    x = layers.BatchNormalization(axis=channel_axis, name="block1_conv1_bn")(x)
    x = layers.Activation("relu", name="block1_conv1_act")(x)
    x = layers.Conv2D(64, (3, 3), use_bias=False, name="block1_conv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block1_conv2_bn")(x)
    x = layers.Activation("relu", name="block1_conv2_act")(x)
    residual = layers.Conv2D(128, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block2_sepconv2_act")(x)
    x = layers.SeparableConv2D(128, (3, 3), padding="same", use_bias=False, name="block2_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block2_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block2_pool")(x)
    x = layers.add([x, residual])
    residual = layers.Conv2D(256, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block3_sepconv1_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block3_sepconv2_act")(x)
    x = layers.SeparableConv2D(256, (3, 3), padding="same", use_bias=False, name="block3_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block3_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block3_pool")(x)
    x = layers.add([x, residual])
    residual = layers.Conv2D(728, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block4_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block4_sepconv2_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block4_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block4_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block4_pool")(x)
    x = layers.add([x, residual])
    # Middle Flow - doesn't change dimensions
    for i in range(8):
        residual = x
        prefix = "block" + str(i + 5)
        x = layers.Activation("relu", name=prefix + "_sepconv1_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv1",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv1_bn")(x)
        x = layers.Activation("relu", name=prefix + "_sepconv2_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv2",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv2_bn")(x)
        x = layers.Activation("relu", name=prefix + "_sepconv3_act")(x)
        x = layers.SeparableConv2D(728,(3, 3),padding="same",use_bias=False,name=prefix + "_sepconv3",)(x)
        x = layers.BatchNormalization(axis=channel_axis, name=prefix + "_sepconv3_bn")(x)
        x = layers.add([x, residual])
    # Exit Flow has The Ability to change Dimensions through strides 
    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2), padding="same", use_bias=False)(x)
    residual = layers.BatchNormalization(axis=channel_axis)(residual)
    x = layers.Activation("relu", name="block13_sepconv1_act")(x)
    x = layers.SeparableConv2D(728, (3, 3), padding="same", use_bias=False, name="block13_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block13_sepconv2_act")(x)
    x = layers.SeparableConv2D(1024, (3, 3), padding="same", use_bias=False, name="block13_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block13_sepconv2_bn")(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding="same", name="block13_pool")(x)
    x = layers.add([x, residual])
    x = layers.SeparableConv2D(1536, (3, 3), padding="same", use_bias=False, name="block14_sepconv1")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv1_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv1_act")(x)
    x = layers.SeparableConv2D(2048, (3, 3), padding="same", use_bias=False, name="block14_sepconv2")(x)
    x = layers.BatchNormalization(axis=channel_axis, name="block14_sepconv2_bn")(x)
    x = layers.Activation("relu", name="block14_sepconv2_act")(x)
    if pooling == "avg":
      x = layers.GlobalAveragePooling2D()(x)
    elif pooling == "max":
      x = layers.GlobalMaxPooling2D()(x)
    inputs = img_input
    # Creating Model
    model = training.Model(inputs, x, name="xception")
    # Giving It weights of Image Net 
    if weights == "imagenet":
        weights_path = data_utils.get_file("xception_weights_tf_dim_ordering_tf_kernels_notop.h5",TF_WEIGHTS_PATH_NO_TOP,cache_subdir="models",file_hash="b0042744bf5b25fce3cb969f33bebb97",)
        model.load_weights(weights_path)
    return model

In [9]:
# Giving The Appropiate arguments
xception=Xception(input_shape=(150,150,3),weights = 'imagenet',include_top=False) 
# Freezing The Trainability of The Layers
# Fine Tunining
for layer in xception.layers:
  layer.trainable=False

83683744/83683744 [==============================] - 1s 0us/step


In [10]:
from keras.layers import Flatten,Dense
x=Flatten()(xception.output)
prediction7=Dense(6,activation='softmax')(x)

In [11]:
from keras.models import Model
# Using Appropiate Optimizer and Loss Function
model7=Model(inputs=xception.input,outputs=prediction7)
model7.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model7.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 74, 74, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                              

In [12]:
# Fitting Model over Data
model7.fit(X_Train,Y_Train,epochs=25)

Epoch 1/25
42/42 [==============================] - 12s 61ms/step - loss: 1.7039 - accuracy: 0.7433
Epoch 2/25
42/42 [==============================] - 3s 61ms/step - loss: 0.2300 - accuracy: 0.9494
Epoch 3/25
42/42 [==============================] - 3s 61ms/step - loss: 0.0538 - accuracy: 0.9844
Epoch 4/25
42/42 [==============================] - 3s 61ms/step - loss: 0.0143 - accuracy: 0.9955
Epoch 5/25
42/42 [==============================] - 3s 61ms/step - loss: 0.0070 - accuracy: 0.9970
Epoch 6/25
42/42 [==============================] - 3s 62ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 7/25
42/42 [==============================] - 3s 62ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 8/25
42/42 [==============================] - 3s 63ms/step - loss: 9.7948e-04 - accuracy: 1.0000
Epoch 9/25
42/42 [==============================] - 3s 63ms/step - loss: 7.8852e-04 - accuracy: 1.0000
Epoch 10/25
42/42 [==============================] - 3s 63ms/step - loss: 6.5278e-04 - accuracy: 1.0

In [13]:
# Evaluation Over Validation Data
model7.evaluate(X_Valid,Y_Valid)

11/11 [==============================] - 2s 128ms/step - loss: 0.6688 - accuracy: 0.9050


[0.6688272953033447, 0.9050444960594177]

In [15]:
# Saving Model 
model7.save('xception-final-Model2.hdf5')